In [2]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import numpy as np
import sklearn

In [3]:
base_url = 'https://krisha.kz/arenda/kvartiry/almaty/?das[rent.period]=2'

In [5]:
headers = {
    'accept':'*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

In [ ]:
flats = []
urls = []
urls.append(base_url)
print(urls)

In [ ]:
session = requests.Session()
request = session.get(base_url, headers = headers)
print(request)

In [ ]:
if request.status_code == 200:
    soup = bs(request.content, 'lxml')
    try:
        pagination = soup.find_all('a', attrs = {'class':'paginator__btn'})
        count = int(pagination[-1].text)
        print(count)
        for i in range(count-1):
            url = f'https://krisha.kz/arenda/kvartiry/almaty/?das[rent.period]=2&page={i+1}'
            if url not in urls:
                urls.append(url)
                print(url)
    except:
        pass

In [ ]:
for url in urls:
        request = session.get(url, headers = headers)
        soup = bs(request.content, 'lxml')

        divs = soup.find_all('div', attrs = {'class':'a-card__inc'})

        for div in divs:
            title = div.find('a', attrs = {'class':'a-card__title'}).text
            price = div.find('div', attrs = {'class':'a-card__price'}).text
            #area = div.find('div', attrs = {'class:offer__advert-short-info'}).text
            href = div.find('a', attrs = {'class':'a-card__title'})['href']
            address = div.find('div', attrs = {'class':'a-card__subtitle'}).text
            content = div.find('div', attrs = {'class':'a-card__text-preview'}).text
            owner = div.find('div', attrs = {'class':'a-card__owner user-title-not-pro'})
            if owner is None:
                specialist = 1
                owner = 0
            else:
                specialist = 0
                owner = 1
            flats.append({
                'title': title,
                'price': price,
                'href': href,
                'address': address,
                'content': content,
                'specialist': specialist,
                'owner': owner
            })
    print(len(flats))
else:
    print('ERROR')
    return flats

In [ ]:
with open(r"Krisha.csv", "w", encoding = 'utf-8') as file:
    a_pen = csv.writer(file)
    a_pen.writerow(('title', 'price', 'href', 'address', 'content','owner','specialist'))
    for flat in flats:
        a_pen.writerow((flat['title'],flat['price'],flat['href'],flat['address'],flat['content'],flat['owner'],flat['specialist']))


In [ ]:
flats = krisha_parse(base_url, headers)

In [ ]:
files_writer(flats)

In [16]:
files_writer(flats)

In [ ]:
df = pd.read_csv('Krisha.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
new = df["title"].str.split(",", expand = True)
df["rooms"]= new[0]
df["square"]= new[1] 
#df["floor"]=new[2]
df.drop(columns =["title"], inplace = True)

new = df["address"].str.split(",", expand = True)
df["district"]= new[0]
df["street"]= new[1]
df.drop(columns =["address"], inplace = True)

new = df["district"].str.split(expand = True)
df["district"] = new[0]

new = df["square"].str.split(expand = True)
df["square"] = new[0]
df['square']=pd.to_numeric(df['square'])

df.drop(columns =["href"], inplace = True)

new = df["price"].str.split(expand = True)
new = new[0]+new[1]
df["price"] = new
df['price']=pd.to_numeric(df['price'])
df.loc[df.price <= 10000,'price'] = df.price * 1000

new = df['rooms'].str.split('-', expand = True)
df['rooms'] = new[0]

df.drop(columns =["street"], inplace = True)

#new = df['floor'].str.split('/', expand = True)
#df['floor'] = new[0]
#new = new.dropna()
#new = new[1].str.split(' ',expand = True)
#df['total floors'] = new[0]

df = df.dropna()

districts = ['Алмалинский','Бостандыкский','Ауэзовский','Медеуский','Жетысуский','Наурызбайский','Алатауский','Турксибский']
df = df[df['district'].isin(districts)]

year = list(df['content'])
yeardf=[]
for years in year:
    years = years[years.find("г.п.")-5:years.find("г.п.")-1]
    yeardf.append(years)
df['year'] = yeardf

df.drop(columns = ["content"], inplace = True)

df['yearisnull'] = df['year'].str.find('    ')
df = df.loc[df.yearisnull != 0]

df.drop(columns =["yearisnull"], inplace = True)

df['rooms']=pd.to_numeric(df['rooms'])
#df = df.loc[df.rooms!=11]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
df1 = df.groupby(['district']).size().reset_index(name='count')
df1.sort_values(by=['count'], inplace=True)
df1

In [ ]:
df1.set_index('district', inplace=True)

In [ ]:
ds.plot(kind = "scatter")

In [ ]:
df1.plot(kind='barh', 
             stacked=False,
             figsize=(20, 10),
             color='g',
             label=df1['count']
             )

plt.title('Распределение арендного жилья в городе Алматы')
plt.xlabel('Количество квартир')
plt.ylabel('Районы')
plt.show()

In [ ]:
df.plot(kind='scatter', x='price', y='square', figsize=(12, 9), color='darkblue')

plt.title('Зависимость стоимости аренды от площади квартиры')
plt.xlabel('Стоимость аренды')
plt.ylabel('Площадь')

x = df['price']
y = df['square']
fit = np.polyfit(x, y, deg=1)
plt.plot(x, fit[0] * x + fit[1], color='red')

plt.show()

In [ ]:
df2 = df.loc[df.price <= 200000]

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.plot(kind='scatter', x='price', y='square', figsize=(12, 9), color='darkblue')

plt.title('Зависимость стоимости аренды от площади квартиры')
plt.xlabel('Стоимость аренды')
plt.ylabel('Площадь')

x = df2['price']
y = df2['square']
fit = np.polyfit(x, y, deg=1)
plt.plot(x, fit[0] * x + fit[1], color='red')

plt.show()

In [ ]:
df4 = df.groupby(['district']).sum()
#df4.sort_values(by=['count'], inplace=True)
df4

In [ ]:
n_groups = 8

# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35

rects1 = plt.barh(index + bar_width, df4['owner'], bar_width,
color='b',
label='Хозяин')

rects2 = plt.barh(index, df4['specialist'], bar_width,
color='g',
label='Риэлтор')

plt.ylabel('Районы')
plt.xlabel('Количество квартир')
plt.title('Распределение арендного жилья в городе Алматы')
plt.yticks(index + bar_width/2, df4.index)
plt.legend()

In [ ]:
df.head()

In [ ]:
df5 = df.groupby(['rooms']).size().reset_index(name='count')
df5.sort_values(by=['count'], inplace=True)
df5.set_index('rooms', inplace=True)
df5

In [ ]:
df5.plot(kind='barh', 
             stacked=False,
             figsize=(20, 10),
             color='g',
             label=df5['count']
             )

plt.title('Распределение арендного жилья в городе Алматы')
plt.xlabel('Количество квартир')
plt.ylabel('Районы')
plt.show()

In [ ]:
df6 = df.groupby(['district','price']).size().reset_index(name='count')
df6.sort_values(by=['count'], inplace=True)
df6 = df6.pivot(index='price',columns='district',values='count')

In [ ]:
df6